In [1]:
%cd ..

d:\Blockchain\data\orai\orchai_validator


In [2]:
from labeling.etl_processor import ETLProcessor
from labeling.tools import get_spark, query, upload
from omegaconf import OmegaConf
import pyspark.sql.functions as F
from labeling.upload import sampling

# Test ETL processor

In [3]:
config = OmegaConf.load("config/etl.yaml")

In [7]:
config.dest

{'host': '103.160.75.28', 'port': 5432, 'user': 'orchai_rw', 'password': 'travaorchai_rw123', 'database': 'orchai', 'table': 'public.block_label', 'batchsize': 1000}

In [5]:
spark = get_spark()

In [8]:
df = query(spark, **config.src)

Successfully queried data from database


In [9]:
df.count()

143687999

In [ ]:
df.repartition(1).write.csv("data/block_label", header=True)

In [10]:
df = sampling(spark, config, from_block=7050501, to_block=7052499)
df.printSchema()

Successfully queried data from database
root
 |-- block_height: integer (nullable = true)
 |-- operator_address: string (nullable = true)
 |-- jailed: boolean (nullable = true)
 |-- status: string (nullable = true)
 |-- tokens: decimal(38,0) (nullable = true)
 |-- commission_rate: float (nullable = true)
 |-- delegator_shares: decimal(38,0) (nullable = true)
 |-- self_bonded: decimal(38,0) (nullable = true)
 |-- propose: boolean (nullable = true)
 |-- vote: boolean (nullable = true)



In [14]:
df.groupBy("block_height").count().count()

1999

In [13]:
df.agg({"block_height": "min"}).show(1)
df.agg({"block_height": "max"}).show(1)

+-----------------+
|min(block_height)|
+-----------------+
|          7070501|
+-----------------+

+-----------------+
|max(block_height)|
+-----------------+
|          7072499|
+-----------------+



In [16]:
df = ETLProcessor.data_scoring(df, **config.hp.etl)

------------------------------------------------
Successfully converted voting_power_score column
------------------------------------------------
------------------------------------------------
Successfully converted commission_score column
------------------------------------------------
------------------------------------------------
Successfully converted self_bonded_score column
------------------------------------------------
------------------------------------------------
Successfully converted vote_propose_score column
------------------------------------------------
------------------------------------------------
Sucessfully converted final_score
------------------------------------------------
------------------------------------------------
Sucessfully combine data
------------------------------------------------
------------------------------------------------
Sucessfully shifting data
------------------------------------------------


In [8]:
df.printSchema()

root
 |-- operator_address: string (nullable = true)
 |-- comission_score: double (nullable = true)
 |-- score: double (nullable = true)
 |-- voting_power_score: double (nullable = true)
 |-- tokens_proportion: decimal(38,10) (nullable = true)
 |-- commission_rate: double (nullable = true)
 |-- self_bonded: decimal(38,4) (nullable = true)
 |-- self_bonded_score: double (nullable = true)
 |-- delegator_shares: decimal(38,4) (nullable = true)
 |-- tokens: decimal(38,4) (nullable = true)
 |-- block_height: integer (nullable = true)
 |-- vote_propose_score: double (nullable = true)
 |-- label: double (nullable = true)



In [17]:
df.groupBy("block_height").count().count()

72

In [ ]:
df.agg({"block_height": "min"}).show(1)
df.agg({"block_height": "max"}).show(1)

In [16]:
(1999 - config.hp.etl.vote_proposed_win_size + 1) / config.hp.etl.combine_win_size - config.hp.etl.label_win_size / config.hp.etl.combine_win_size

72.0

## => correct

## Tuning batch_size

In [ ]:
vote_proposed_win_size = config.hp.etl.vote_proposed_win_size
combine_win_size = config.hp.etl.combine_win_size
label_win_size = config.hp.etl.label_win_size

batch_size = 150000
batch_size = (batch_size - vote_proposed_win_size + 1) // combine_win_size * combine_win_size + vote_proposed_win_size - 1
print("new batch size =", batch_size)
print("number of transformed blocks =", (batch_size - vote_proposed_win_size + 1) / combine_win_size - label_win_size // combine_win_size)